# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [32]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
from selenium.webdriver.chrome.service import Service 
import os

In [3]:
my_service = Service(executable_path=r'C:\Users\Chokfi\Downloads\chromedriver_win32\chromedriver.exe')

browser = Browser('chrome', service=my_service)

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [4]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = 'https://static.bc-edx.com/data/web/mars_facts/temperature.html'
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [5]:
# Create a Beautiful Soup Object

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [6]:
# Extract all rows of data
table = soup.find('table', class_='table')

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [14]:
# Create an empty list
column_labels = []
labels = table.find_all('th')
for title in labels:
    column_labels.append(title.text)
    

df_rows = []
# Loop through the scraped data to create a list of rows
rows = table.find_all('tr', class_='data-row')
for row in rows:
    rw_ls = row.find_all('td')
    table_row = []
    for rw in rw_ls:
        table_row.append(rw.text)
    df_rows.append(table_row)
    


In [15]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names
mars_df = pd.DataFrame(df_rows, columns = column_labels)

In [16]:
# Confirm DataFrame was created successfully
mars_df.head()

,id,terrestrial_date,sol,ls,month,min_temp,pressure
0,2,2012-08-16,10,155,6,-75.0,739.0
1,13,2012-08-17,11,156,6,-76.0,740.0
2,24,2012-08-18,12,156,6,-76.0,741.0
3,35,2012-08-19,13,157,6,-74.0,732.0
4,46,2012-08-20,14,157,6,-74.0,740.0


### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [18]:
# Examine data type of each column
mars_df.dtypes

id                  object
terrestrial_date    object
sol                 object
ls                  object
month               object
min_temp            object
pressure            object
dtype: object

In [37]:
# Change data types for data analysis
mars_df = mars_df.astype({'id':'int64'})
mars_df = mars_df.astype({'ls':'float64'})
mars_df = mars_df.astype({'month':'int64'})
mars_df = mars_df.astype({'sol':'int64'})
mars_df = mars_df.astype({'min_temp':'float64'})
mars_df = mars_df.astype({'pressure':'float64'})
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'])

In [38]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

id                           int64
terrestrial_date    datetime64[ns]
sol                          int64
ls                         float64
month                        int64
min_temp                   float64
pressure                   float64
dtype: object

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [39]:
# 1. How many months are there on Mars?
month_num = mars_df['month'].max()
month_num

12

In [43]:
# 2. How many Martian days' worth of data are there?
days_num = mars_df['sol'].count()
print(days_num)


1867


In [45]:
# 3. What is the average low temperature by month?
mon_mars_df = mars_df.groupby(['month']).mean()
mon_mars_df['min_temp']

month
1    -77.160920
2    -79.932584
3    -83.307292
4    -82.747423
5    -79.308725
6    -75.299320
7    -72.281690
8    -68.382979
9    -69.171642
10   -71.982143
11   -71.985507
12   -74.451807
Name: min_temp, dtype: float64

In [ ]:
# Plot the average temperature by month


In [49]:
# Identify the coldest and hottest months in Curiosity's location
cold = mon_mars_df['min_temp'].min()
hot = mon_mars_df['min_temp'].max()

cold_mon = mon_mars_df.index[mon_mars_df['min_temp']==cold]
hot_mon = mon_mars_df.index[mon_mars_df['min_temp']==hot]

print(cold_mon)
print(hot_mon)

Int64Index([3], dtype='int64', name='month')
Int64Index([8], dtype='int64', name='month')


In [46]:
# 4. Average pressure by Martian month
mon_mars_df['pressure']

month
1     862.488506
2     889.455056
3     877.322917
4     806.329897
5     748.557047
6     745.054422
7     795.105634
8     873.829787
9     913.305970
10    887.312500
11    857.014493
12    842.156627
Name: pressure, dtype: float64

In [51]:
low = mon_mars_df['pressure'].min()
high = mon_mars_df['pressure'].max()

low_mon = mon_mars_df.index[mon_mars_df['pressure']==low]
high_mon = mon_mars_df.index[mon_mars_df['pressure']==high]

print(low_mon)
print(high_mon)


Int64Index([6], dtype='int64', name='month')
Int64Index([9], dtype='int64', name='month')


In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?


On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [35]:
# Write the data to a CSV
mars_df.to_csv('mars.csv', index=False, encoding='utf-8')

In [50]:
browser.quit()